In [1]:
# https://huggingface.co/lllyasviel/control_v11p_sd15_openpose

In [2]:
import torch
import os
import numpy as np
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from controlnet_aux import OpenposeDetector
from PIL import Image

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image

def generate_pose_map(image_path, openpose_model):
    image = load_image(image_path)
    pose_map = openpose_model(image)
    return pose_map

def pose_transfer(original_image_path, controlnet_pipe, openpose_model):
    pose_map = generate_pose_map(original_image_path, openpose_model)
    
    # Convert pose map to PIL Image if needed
    if isinstance(pose_map, np.ndarray):
        pose_map = Image.fromarray(pose_map)
    
    # Generate new image with the given pose
    result = controlnet_pipe(prompt="A person sitting in comfortable clothes.", image=pose_map).images[0]
    return result

c:\Users\010\anaconda3\envs\catvton\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\010\AppData\Roaming\Python\Python39\site-packages\controlnet_aux\mediapipe_face\mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [4]:
from datetime import datetime
# Load ControlNet model
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
pipe.to("cuda")

# Load OpenPose model
openpose_model = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

# Input image path

input_image_path = "../sample_img/standing_sample/standing_01.jpg"
output_image = pose_transfer(input_image_path, pipe, openpose_model)

# 기본 설정값 (변수 선언)
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 결과 이미지 저장
date_str = datetime.now().strftime("%Y%m%d%H%M%S")
folder_path = os.path.join(output_dir, date_str[:8])
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
result_save_path = os.path.join(folder_path, date_str[-6:] + ".jpg")

# Save result
output_image.save(result_save_path)

An error occurred while trying to fetch lllyasviel/sd-controlnet-openpose: lllyasviel/sd-controlnet-openpose does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


OSError: lllyasviel/sd-controlnet-openpose does not appear to have a file named diffusion_pytorch_model.bin.

예제

In [3]:
import torch
import os
#from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image
import numpy as np
from controlnet_aux import OpenposeDetector

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

from datetime import datetime
# 기본 설정값 (변수 선언)
output_dir = "./output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 결과 이미지 저장
date_str = datetime.now().strftime("%Y%m%d%H%M%S")
folder_path = os.path.join(output_dir, date_str[:8])
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

checkpoint = "lllyasviel/control_v11p_sd15_openpose"

# image = load_image(
#     "https://huggingface.co/lllyasviel/control_v11p_sd15_openpose/resolve/main/images/input.png"
# )

prompt = "A person sitting in a wheelchair wearing comfortable clothes."
processor = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')


# Input image path
image_path = "../sample_img\sitting_sample\sample03.jpg"
image = Image.open(image_path).convert("RGB")

control_image = processor(image, hand_and_face=True)

control_save_path = os.path.join(folder_path, "pose_" + date_str[-6:] + ".jpg")

control_image.save(control_save_path)

controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

generator = torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=30, generator=generator, image=control_image).images[0]

result_save_path = os.path.join(folder_path, "sitting_" + date_str[-6:] + ".jpg")
image.save(result_save_path)

100%|██████████| 30/30 [00:05<00:00,  5.79it/s]


stable diffusion sdxl

In [4]:
import torch
from diffusers import StableDiffusionXLPipeline

# Stable Diffusion XL 모델 로드 (fp16을 사용하면 VRAM 절약 가능)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16, 
    use_safetensors=True
).to("cuda")  # GPU 사용

# 텍스트 프롬프트 설정
prompt = "A person sitting in a wheelchair wearing comfortable clothes, highly detailed, realistic, 4K"

# 이미지 생성
image = pipe(prompt).images[0]

# 이미지 저장 및 표시
result_save_path = os.path.join(folder_path, "wheelchair_person_" + date_str[-6:] + ".jpg")
image.save(result_save_path)
image.show()
# image.save("wheelchair_person.png")
# image.show()

c:\Users\010\anaconda3\envs\catvton\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\010\.cache\huggingface\hub\models--stabilityai--stable-diffusion-xl-base-1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 19 files:  26%|██▋       | 5/19 [56:27<2:38:06, 677.58s/it]


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None)", ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))